In [26]:
from bokeh.io import push_notebook,show,output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, OpenURL, TapTool, CustomJS,BoxSelectTool,Rect,Ellipse
import numpy as np
output_notebook()

Loading BokehJS ...

In [27]:
import matplotlib.image as mpimg
import scipy
file_name='./chess.jpg'
#file_name='./data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'
#file_name='./data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
#file_type=255
file_type=1
img2=mpimg.imread(file_name)*file_type
alpha=np.ones((img2.shape[0],img2.shape[1],1))*255
img2=np.concatenate([img2[:,:,:3],alpha],axis=2)
x=[]
y=[]
x_rand=[]
y_rand=[]
probs=[]
#img2=scipy.misc.imresize(img2,(img2.shape[0]/2,img2.shape[1]/2))
img2.shape

(500, 500, 4)

In [36]:

radius=4
source = ColumnDataSource(data=dict(x=x, y=y))
x_var='x'
y_var='y'

callback = CustomJS(args=dict(source=source), code="""
        // get data source from Callback args
        var data = source.data;
        console.log("hola");
        console.log(cb_obj);
        
        /// update data source with new Rect attributes
        data['x'].push(cb_obj.x);
        data['y'].push(cb_obj.y+%d);

        // emit update of data source
        
        if (IPython.notebook.kernel !== undefined) {
            var kernel = IPython.notebook.kernel;
            cmd = "%s.append(" + cb_obj.x+")";
            kernel.execute(cmd, {}, {});
            cmd = "%s.append(" + cb_obj.y+")";
            kernel.execute(cmd, {}, {});
        }
        console.log(source);
        source.change.emit();
    """%(0,x_var,y_var))

#box_select = TapTool(callback=callback)

N1=img2.shape[0]
N2=img2.shape[1]
p3=figure(x_range=(0,N2), y_range=(0,N1),plot_width=int(N2/1.5),plot_height=int(N1/1.5))#,tools=[box_select])
p3.js_on_event('tap', callback)
img2[0:20,0:20,0:3]=0
p3.image_rgba([np.array(img2,dtype=np.uint8)],x=0, y=0, dw=N2, dh=N1)
el = Ellipse(x='x',
            y='y',
            width=radius*2,
            height=radius*2,
            line_alpha=1,
            fill_alpha=1,
            line_color='#FF0000',
            fill_color='#FF0000')
p3.add_glyph(source, el, selection_glyph=el, nonselection_glyph=el)

source2 = ColumnDataSource(
    data=dict(x=x_rand, y=y_rand,alpha=probs))
el2 = Ellipse(x='x',
            y='y',
            width=radius*2,
            height=radius*2,
            line_alpha='alpha',
            # line_alpha=0.5,
            fill_alpha='alpha',
            #  fill_alpha=0.5,
            line_color='#00FF00',
            fill_color='#00FF00')
p3.add_glyph(source2, el2, selection_glyph=el2, nonselection_glyph=el2)

t2=show(p3,notebook_handle=True)

In [40]:
print probs.shape
x_rand=x_rand[probs>0.5]
y_rand=y_rand[probs>0.5]
probs=probs[probs>0.5]
probs[probs>0.5].shape,probs.shape
clusters=np.ones((probs.shape[0],))*-1                  
for i in range(probs.shape[0]):
    


(11910,)


array([-1., -1., -1., ..., -1., -1., -1.])

In [7]:
import json
f=open(file_name.replace('.png','.json').replace('.jpg','.json'),'w')
f.write(json.dumps({'x':x,'y':y}))
f.close()


In [29]:
import json
f=open(file_name.replace('.png','.json').replace('.jpg','.json'),'r')
d=json.loads(f.read())
x=d['x']
y=d['y']


In [135]:
x.pop()
y.pop()

386.63956361784886

In [88]:
import numpy as np
np.random.randint(0,2,size=(3,3))

array([[0, 0, 1],
       [0, 0, 0],
       [1, 1, 0]])

In [32]:
x_arr=np.array(x)
y_arr=np.array(y)
M=100000
im_size=40
stride=im_size/2
dist_max=im_size/4
x_rand=np.random.randint(int(stride),N2-int(stride),size=M)
y_rand=np.random.randint(int(stride),N1-int(stride),size=M)
dists=np.zeros(M)
for i in range(M):
    dists[i]=np.min(((x_arr-x_rand[i])**2+(y_arr-y_rand[i])**2)**0.5)
    
probs=(dist_max - np.min(np.concatenate([dists.reshape(M,1),np.ones((M,1))*dist_max],axis=1),axis=1))>0#/float(dist_max)
print probs
prob_select=(probs==0.)*np.random.randint(0,100,size=M)
print prob_select
FILTER=15
probs=probs[prob_select<FILTER]
x_rand=x_rand[prob_select<FILTER]
y_rand=y_rand[prob_select<FILTER]
M=x_rand.shape[0]
print np.mean(probs)
print np.sum(prob_select<FILTER)
print x_rand.shape
#probs

[False False False ..., False False  True]
[62 69 10 ..., 34 68  0]
0.474256361247
25113
(25113,)


In [33]:
x_list=[]
y_list=[]
y_rand2=y_rand
for i in range(M):
    im=img2[y_rand2[i]-int(stride):y_rand2[i]+int(stride),x_rand[i]-int(stride):x_rand[i]+int(stride),:3].reshape(1,im_size,im_size,3).astype(np.float64)
    im/=255.
    x_list.append(im)
    y_list.append(probs[i].reshape(1,1))
X=np.concatenate(x_list)
Y=np.concatenate(y_list).reshape(M,1)
Y=np.concatenate([Y,1-Y],axis=1)
X.shape,Y.shape


((25113, 40, 40, 3), (25113, 2))

In [6]:
batch_size = 200 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 10 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.5 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 3000 # the FC layer will have 512 neurons

In [18]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import keras
import numpy as np
inp = Input(shape=(40, 40, 3)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax

'''flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(2, activation='softmax')(drop_3)'''

conv_5= Convolution2D(3000,(10,10),strides=(1,1),padding='valid',activation='relu')(drop_2)
drop_3= Dropout(drop_prob_1)(conv_5)
conv_6=Convolution2D(2,(1,1),padding='valid',activation='softmax')(drop_3)
out=Flatten()(conv_6)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer=adam, # using the Adam optimiser
              metrics=['accuracy']) 
model.summary()
model.layers

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 40, 40, 3)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 40, 40, 32)        9632      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 40, 40, 32)        102432    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 20, 20, 64)        204864    
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 20, 20, 64)        409664    
__________

In [46]:
model.fit(X, Y,                # Train the model using the training set...
          batch_size=batch_size, epochs=10,
          verbose=1, validation_split=0.1) 

Train on 22566 samples, validate on 2508 samples
Epoch 1/10
22566/22566 [==============================] - 7s - loss: 0.0639 - acc: 0.9756 - val_loss: 0.0567 - val_acc: 0.9793
Epoch 2/10
22566/22566 [==============================] - 7s - loss: 0.0609 - acc: 0.9768 - val_loss: 0.0544 - val_acc: 0.9833
Epoch 3/10
22566/22566 [==============================] - 7s - loss: 0.0596 - acc: 0.9773 - val_loss: 0.0597 - val_acc: 0.9777
Epoch 4/10
22566/22566 [==============================] - 7s - loss: 0.0595 - acc: 0.9760 - val_loss: 0.0579 - val_acc: 0.9809
Epoch 5/10
22566/22566 [==============================] - 7s - loss: 0.0590 - acc: 0.9770 - val_loss: 0.0536 - val_acc: 0.9817
Epoch 6/10
22566/22566 [==============================] - 7s - loss: 0.0596 - acc: 0.9758 - val_loss: 0.0523 - val_acc: 0.9809
Epoch 7/10
22566/22566 [==============================] - 7s - loss: 0.0591 - acc: 0.9772 - val_loss: 0.0562 - val_acc: 0.9809
Epoch 8/10
22566/22566 [==============================] - 7s -

In [78]:
model.save_weights('conv_chess.h5')

In [19]:
model.load_weights('conv_chess.h5')

In [151]:
model.evaluate(X,Y)

59424/59462 [============================>.] - ETA: 0s

[0.092308334196224306, 0.97934815512428108]

In [20]:
probs=model.predict(X,verbose=1)[:,0]>0.95

25214/25214 [==============================] - 4s     


In [11]:
conv_out=125
probs=model.predict((img2[:,:,:3]/255.).reshape(1,N1,N2,3)).reshape(conv_out,conv_out,2)[:,:,0].reshape(-1)>0.9

x_rand=[]
y_rand=[]
for k in range(conv_out):
    for u in range(conv_out):
        y_rand.append(4*u)
        x_rand.append(4*k)
        

In [20]:
i=12
im_total=np.zeros((400,400,4))
p4=figure(x_range=(0,400), y_range=(0,400),plot_width=400,plot_height=400)#,tools=[box_select])
for i in range(10):
    for j in range(10):        
        im=(np.concatenate([X[i*10+j],np.ones((40,40,1),dtype=np.float64)],axis=2)*255)
        im_total[i*40:(i+1)*40,j*40:(j+1)*40,:]=im
        
        
print im.shape
p4.image_rgba([im_total.astype(np.uint8)],x=0, y=0, dw=400, dh=400)
for i in range(10):
    for j in range(10): 
        yi=Y[i*10+j,0]
        #print y
        p4.ellipse(x=j*40+20,
            y=i*40+20,
            width=10,
            height=10,
            line_alpha=yi,
            fill_alpha=yi,
            line_color='#00FF00',
            fill_color='#00FF00')
t2=show(p4,notebook_handle=True)
y_rand[0],x_rand[0]




(40, 40, 4)


(60, 344)